In [134]:
import numpy as np
import pandas as pd

In [135]:
df = pd.read_excel("Bond Data/ftse_balance_prices.xlsx", sheet_name="Sheet1")

print(df.head(3))
print(df.tail(3))

        Date  Last Price
0 2025-05-28         NaN
1 2025-05-27     5578.73
2 2025-05-26     5526.75
           Date  Last Price
1297 2020-06-02     4422.08
1298 2020-06-01     4402.58
1299 2020-05-29     4393.36


In [136]:
# 1. Total Price Return over the full period

start_date = df['Date'].iloc[-1]
end_date = df['Date'].iloc[1]

P_start = df['Last Price'].iloc[-1]
P_end = df['Last Price'].iloc[1]

total_return = P_end / P_start - 1

print(f"Total Price Return: {total_return:.2%}")

print("Start Date:", start_date.date(), "P_start:", P_start)
print("End Date:", end_date.date(), "P_end:", P_end)



Total Price Return: 26.98%
Start Date: 2020-05-29 P_start: 4393.36
End Date: 2025-05-27 P_end: 5578.73


In [137]:
# 2. CAGR over n years

days = (df['Date'].iloc[1] - df['Date'].iloc[-1]).days

n_years = days / 365.25

cagr = (P_end / P_start)**(1 / n_years) - 1

print(f"CAGR over 5 years: {cagr:.2%}")



CAGR over 5 years: 4.90%


In [138]:
# 3. (Optional) Dividend-Adjusted CAGR
# If you have an estimated average dividend yield, e.g. 2.2%:
#div_yield = 0.022
#div_adj_cagr = cagr + div_yield
#print(f"Dividend-Adjusted CAGR ≈ {div_adj_cagr:.2%}")



In [139]:
# 4. Annualised Volatility
# Annualised volatility is the standard deviation of daily returns, scaled by the square root of 252 trading days.

# First get daily return
df = df.sort_values('Date').reset_index(drop=True)
rets = df['Last Price'].pct_change().dropna()

# Count observations because they are not 252
n_obs = len(rets)
years = (end_date - start_date).days / 365.25
print(f"Number of observations: {n_obs}, Years: {years:.2f}")

trading_days = n_obs / years
print(f"Trading Days: {trading_days:.2f}")

# Calucate annualised volatility
daily_std = rets.std()
annualised_volatility = daily_std * np.sqrt(trading_days)
print(f"Annualised Volatility: {annualised_volatility:.2%}")





Number of observations: 1299, Years: 4.99
Trading Days: 260.12
Annualised Volatility: 7.76%


/var/folders/k0/9514c2kj447cm58pbj7ncbpw0000gn/T/ipykernel_23451/944100849.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = df['Last Price'].pct_change().dropna()


In [140]:
# 5. Max Drawdown
# compute running maximum of price series
running_max = df['Last Price'].cummax().iloc[1:] # returns the maximum value of the series up to each point

# Then, we calculate the drawdown at each date
drawdown = df['Last Price'] / running_max - 1

# The maximum drawdown is the minimum value of the drawdown series
max_drawdown = drawdown.min()
print(f"Max Drawdown: {max_drawdown:.2%}")

# Locating the date of the maximum drawdown
trough_max_drawdown = drawdown.idxmin()
trough_date = df.loc[trough_max_drawdown, 'Date']
print(f"Max Drawdown Date: {trough_date.date()}")

# Finding the peak before the trough
peak_idx = df['Last Price'][:trough_max_drawdown].idxmax()
peak_date = df.loc[peak_idx, 'Date']

print(f"Max Drawdown Date: {max_drawdown:.2%}")
print(f"Peak Date (Start of drop): {peak_date.date()} at Price: {df.loc[peak_idx, 'Last Price']:.2f}")
print(f"Trough Date (End of drop): {trough_date.date()} at Price: {df.loc[trough_max_drawdown, 'Last Price']:.2f}")


Max Drawdown: -11.02%
Max Drawdown Date: 2022-06-16
Max Drawdown Date: -11.02%
Peak Date (Start of drop): 2021-12-08 at Price: 5264.22
Trough Date (End of drop): 2022-06-16 at Price: 4684.14


In [141]:
# 6. Sharpe Ratio (r_f = 0)
# annualised return and vol from daily series

mu_daily = rets.mean()  # daily mean return
sigma_daily = rets.std() # daily standard deviation


annual_return = mu_daily * trading_days # E[R], annualised return
annual_vol = sigma_daily * np.sqrt(trading_days) # E[σ], annualised volatility

sharpe = annual_return / annual_vol

print(f"Annualised Return: {annual_return:.2%}")
print(f"Annualised Volatility: {annual_vol:.2%}")
print(f"Sharpe Ratio (rf=0): {sharpe:.2f}")

Annualised Return: 5.08%
Annualised Volatility: 7.76%
Sharpe Ratio (rf=0): 0.66


In [142]:
print("Start Date:", start_date.date(), "Price:", P_start)
print("End Date:", end_date.date(), "Price:", P_end)

print(f"Total Price Return: {total_return:.2%}")

print(f"CAGR over 5 years: {cagr:.2%}")

print(f"Number of observations: {n_obs}, Years: {years:.2f}")
print(f"Trading Days: {trading_days:.2f}")
print(f"Annualised Volatility: {annualised_volatility:.2%}")


print(f"Max Drawdown Date: {max_drawdown:.2%}")
print(f"Peak Date (Start of drop): {peak_date.date()} at Price: {df.loc[peak_idx, 'Last Price']:.2f}")
print(f"Trough Date (End of drop): {trough_date.date()} at Price: {df.loc[trough_max_drawdown, 'Last Price']:.2f}")



print(f"Annualised Return: {annual_return:.2%}")
print(f"Annualised Volatility: {annual_vol:.2%}")
print(f"Sharpe Ratio (rf=0): {sharpe:.2f}")

Start Date: 2020-05-29 Price: 4393.36
End Date: 2025-05-27 Price: 5578.73
Total Price Return: 26.98%
CAGR over 5 years: 4.90%
Number of observations: 1299, Years: 4.99
Trading Days: 260.12
Annualised Volatility: 7.76%
Max Drawdown Date: -11.02%
Peak Date (Start of drop): 2021-12-08 at Price: 5264.22
Trough Date (End of drop): 2022-06-16 at Price: 4684.14
Annualised Return: 5.08%
Annualised Volatility: 7.76%
Sharpe Ratio (rf=0): 0.66


In [143]:
print("🔍 FTSE BALANCE - PERFORMANCE SUMMARY")
print("=" * 50)

print("\n📅 Period Analysis:")
print(f"   Start: {start_date.date()} (£{P_start})")
print(f"   End: {end_date.date()} (£{P_end})")
print(f"   Duration: {years:.2f} years ({n_obs} observations)")

print("\n📊 Returns:")
print(f"   Total Return: {total_return:.2%}")
print(f"   CAGR: {cagr:.2%}")
print(f"   Annualised Return: {annual_return:.2%}")

print("\n⚠️ Risk Analysis:")
print(f"   Volatility: {annualised_volatility:.2%}")
print(f"   Max Drawdown: {max_drawdown:.2%}")
print(f"   Sharpe Ratio: {sharpe:.2f}")

print("\n📉 Worst Period:")
print(f"   Peak: {peak_date.date()} (£{df.loc[peak_idx, 'Last Price']:.2f})")
print(f"   Trough: {trough_date.date()} (£{df.loc[trough_max_drawdown, 'Last Price']:.2f})")


🔍 FTSE BALANCE - PERFORMANCE SUMMARY

📅 Period Analysis:
   Start: 2020-05-29 (£4393.36)
   End: 2025-05-27 (£5578.73)
   Duration: 4.99 years (1299 observations)

📊 Returns:
   Total Return: 26.98%
   CAGR: 4.90%
   Annualised Return: 5.08%

⚠️ Risk Analysis:
   Volatility: 7.76%
   Max Drawdown: -11.02%
   Sharpe Ratio: 0.66

📉 Worst Period:
   Peak: 2021-12-08 (£5264.22)
   Trough: 2022-06-16 (£4684.14)
